Convert LIDO *XML* of SMB Data to CSV and extract fields

Author: Christopher Pietsch


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cp /content/drive/My\ Drive/SMB/Daten/Lido-Export\(24-03-2020\).zip /content/export.zip

In [ ]:
!unzip -q export.zip -d /content/

In [ ]:
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import re

ns = {'lido': 'http://www.lido-schema.org'}
path = "/content/Lido-Export(24-03-2020)/**/*.xml"

def get(tree, path):
  return "; ".join([item.text for item in tree.findall(path, ns)])

columns = ["id", "idlong", "sammlung", "sammlunglong", "fotograf", "museum", "idnr", "titel", "sachbegriff", "actors", "ort", "datum", "material", "abmessung", "beschreibung"]
rows = []

for file in glob.glob(path):
#file = glob.glob(path)[0]

  sammlung = "ang" if "ANG" in file else "mek"
  tree = ET.parse(file)
  idlong = get(tree, './/lido:lidoRecID')
  id = idlong.split("/")[1]
  idnr = get(tree, './/lido:workID')
  titel = get(tree, './/lido:titleSet/lido:appellationValue')
  sachbegriff = get(tree, './/lido:objectWorkType/lido:term')
  actors = get(tree, './/lido:displayActorInRole')
  ort = get(tree, './/lido:displayPlace')
  datum = get(tree, './/lido:displayDate')
  material = get(tree, './/lido:displayMaterialsTech')
  abmessung = get(tree, './/lido:displayObjectMeasurements')
  beschreibung = get(tree, './/lido:descriptiveNoteValue')
  sammlunglong = get(tree, './/lido:administrativeMetadata/**/lido:legalBodyName/lido:appellationValue')
  cc = [item.text for item in tree.findall('.//lido:administrativeMetadata/lido:resourceWrap/lido:resourceSet/*/lido:legalBodyName/lido:appellationValue', ns)]
  if(len(cc) == 1):
    museum = cc[0]
    fotograf = ""
  else:
    fotograf = cc[0]
    museum = cc[1]
  lizenz = get(tree, './/lido:administrativeMetadata/lido:resourceWrap/lido:resourceSet/lido:rightsResource/lido:rightsType/lido:term')
  lizenzurl = get(tree, './/lido:administrativeMetadata/lido:resourceWrap/lido:resourceSet/lido:rightsResource/lido:rightsType/lido:conceptID')

  
  #print(lizenzurl)
  rows.append((id, idlong, sammlung, sammlunglong, fotograf, museum,  idnr, titel, sachbegriff, actors, ort, datum, material, abmessung, beschreibung))

export_df = pd.DataFrame.from_records(rows, columns=columns)

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    print(pd.DataFrame.from_records(rows, columns=["datum", "clean"]))


In [ ]:
export_df

,id,idlong,sammlung,sammlunglong,fotograf,museum,lizenz,lizenzurl,idnr,titel,sachbegriff,actors,ort,datum,material,abmessung,beschreibung
0,966933,\tDE-MUS-815114/966933,ang,"Alte Nationalgalerie, Staatliche Museen zu Berlin",Jörg P. Anders,"Alte Nationalgalerie, Staatliche Museen zu Berlin",Attribution-NonCommercial-ShareAlike 3.0 Unpor...,http://creativecommons.org/licenses/by-nc-sa/3.0/,A II 402,Estaminet. Flämische Schenke,Bild,"Lesser Ury (7.11.1861 - 18.10.1931, Maler)",,1884,Öl auf Leinwand,"Höhe x Breite: 100,5 x 52 cm; Rahmenmaß: 127 x...",Von 1882 bis 1884 lebte Lesser Ury in dem fläm...
1,962560,\tDE-MUS-815114/962560,ang,"Alte Nationalgalerie, Staatliche Museen zu Berlin",Andres Kilger,"Alte Nationalgalerie, Staatliche Museen zu Berlin",Attribution-NonCommercial-ShareAlike 3.0 Unpor...,http://creativecommons.org/licenses/by-nc-sa/3.0/,W.S. 63,Schleichhändler-Felouke vor der Küste von Biskaya,Bild & Pendant,"Théodore Gudin (15.8.1802 - 12.4.1880, Maler)",,1845,Öl auf Leinwand,"Höhe x Breite: 47,2 x 39,3 cm; Rahmenmaß: 62 x...","Théodore Gudin, der in den dreißiger Jahren me..."
2,961278,\tDE-MUS-815114/961278,ang,"Alte Nationalgalerie, Staatliche Museen zu Berlin",Andres Kilger,"Alte Nationalgalerie, Staatliche Museen zu Berlin",Attribution-NonCommercial-ShareAlike 3.0 Unpor...,http://creativecommons.org/licenses/by-nc-sa/3.0/,A III 566,Träumerei,Bild,"Ludwig von Hofmann (17.8.1861 - 23.8.1945, Maler)",,1898,Öl auf Holz,"Höhe x Breite: 45 x 70,5 cm",Die elegische Traumstimmung und zahlreiche sym...
3,967338,\tDE-MUS-815114/967338,ang,"Alte Nationalgalerie, Staatliche Museen zu Berlin",Andres Kilger,"Alte Nationalgalerie, Staatliche Museen zu Berlin",Attribution-NonCommercial-ShareAlike 3.0 Unpor...,http://creativecommons.org/licenses/by-nc-sa/3.0/,A I 569,Sommerabend bei Scheveningen,Bild,"Hendrik Willem Mesdag (23.2.1831 - 10.7.1915, ...",,1896,Öl auf Leinwand,Höhe x Breite: 140 x 180 cm; Rahmenmaß: 188 x ...,Vermutlich ist das große Seebild für die Inter...
4,960472,\tDE-MUS-815114/960472,ang,"Alte Nationalgalerie, Staatliche Museen zu Berlin",Jörg P. Anders,"Alte Nationalgalerie, Staatliche Museen zu Berlin",Attribution-NonCommercial-ShareAlike 3.0 Unpor...,http://creativecommons.org/licenses/by-nc-sa/3.0/,A III 816,Enzwehr bei Besigheim (Studie),Bild,"Gustav Schönleber (3.12.1851 - 1.2.1917, Maler)",,1883,Öl auf Leinwand,"Höhe x Breite: 59 x 73,5 cm","Charakteristisch für Besigheim, die Stadt an N..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5137,501573,DE-MUS-018719/501573,mek,"Museum Europäischer Kulturen, Staatliche Musee...",Fotograf unbekannt,"Museum Europäischer Kulturen, Staatliche Musee...",Attribution-NonCommercial-ShareAlike 3.0 Unpor...,http://creativecommons.org/licenses/by-nc-sa/3.0/,D (52 P 1) 246/1966,Thorazeiger,Thorazeiger,"Rudolf Wissell (9.3.1869 - 13.12.1962, Sammler)",Gebrauchsort: unbekannt; Erwerbungsort: Berlin,wohl nach 1870,"Silber, gegossen und graviert",Länge x Breite: 23 x 1 cm,Der geheiligte Text der Tora darf nicht mit de...
5138,510891,DE-MUS-018719/510891,mek,"Museum Europäischer Kulturen, Staatliche Musee...",Fotograf unbekannt,"Museum Europäischer Kulturen, Staatliche Musee...",Attribution-NonCommercial-ShareAlike 3.0 Unpor...,http://creativecommons.org/licenses/by-nc-sa/3.0/,II B 6392,Romanze,Bilderbogen o. Nr.; Lubok; russischer Volksbil...,Lithographische Werkstatt A. V. Morosov (Verle...,"Herstellungsort: Moskau, Russland",1889,"Lithografie, mit Farbkleckstechnik koloriert","Höhe x Breite: 34,2 x 42,5 cm",Querformat. Romanze (im Stil unserer Küchenlie...
5139,1023347,DE-MUS-018719/1023347,mek,"Museum Europäischer Kulturen, Staatliche Musee...",Fotograf unbekannt,"Museum Europäischer Kulturen, Staatliche Musee...",Attribution-NonCommercial-ShareAlike 3.0 Unpor...,http://creativecommons.org/licenses/by-nc-sa/3.0/,"D (53 Y 150) 1188/1988,a-b",Weihnachtsbaumanhänger aus Papier (2),Weihnachtsbaumanhänger aus Papier (2),,Herkunft (Allgemein): Deutsc

In [ ]:
export_df.to_csv("export2010.csv", index=False)

In [ ]:
!cp export2010.csv /content/drive/My\ Drive/SMB/Daten/clean/

Optional: calculate a clean year for each object

In [ ]:
def calculateYear(datum):
  arr =  [int(i) for i in re.findall(r'\d+', datum)]
  out = 0

  if(len(arr) is 0):
    out = 0
  elif(len(arr) is 1):
    if(arr[0] > 1000):
      out = arr[0]
    else:
      out = arr[0] * 100
  elif(len(arr) is 2):
    if(arr[0] > 1000 and arr[1] > 1000):
      out = arr[0] + int((arr[1] - arr[0])/2)
    elif((arr[0] is 18 or arr[0] is 19) and (arr[1] is 18 or arr[1] is 19 or arr[1] is 20)):
      out = arr[0] * 100
    elif(arr[0] < 10 and arr[1] < 100): #[2, 19] 2. Hälfte 19. Jahrhundert
      out = (arr[1]-1) * 100 + arr[0]* 25
    elif(arr[1] > 1000):
      out = arr[1]
    elif(arr[0] > 1000 and arr[1] < 1000):
      year2 = int(arr[0] / 100) * 100 + arr[1]
      out = arr[0] + int((year2 - arr[0])/2)
      #print(arr, datum, out)
  elif(len(arr) is 3):
    if(arr[0] > 1000):
      out = arr[0]
    elif(arr[2] > 1000):
      out = arr[2]
    else:
      out = 1800
  else:
    if(arr[0] > 1000):
      out = arr[0]
    else:
      print(arr, datum)

  return out

In [ ]:
for datum in export_df['datum'].values:
  clean = calculateYear(datum)
  #print(clean)
  

In [ ]:
export_df['datum_clean'] = [calculateYear(i) for i in export_df['datum'].values]